# Ciminion2 Polynomial Model Demonstration
Usage example of the Ciminion_2 polynomial model.

In [1]:
using Oscar
include("Ciminion_2.jl")
include("Ciminion_2_polynomial_model.jl")

generate_Ciminion_2_polynomials (generic function with 1 method)

## Ciminion2 Instance

In [2]:
K = GF(10007)
rounds_C = 3
rounds_E = 3

ciminion_2 = Ciminion_2_constructor(field=K, 
                                    rounds_C=rounds_C, 
                                    rounds_E=rounds_E);

Ciminion parameters
Field: Prime field of characteristic 10007
Rounds_C: 3
Rounds_E: 3
Constants_C: [6130 2318 4937; 1427 7078 5005; 1602 1151 8506; 4368 8330 5867]
Constants_E: [5255 6676 5726; 7388 5660 507; 8110 9 5962; 5361 4875 2989]


In [3]:
plain = zero_matrix(K, 2, 1)
plain[1, 1] = rand(K)
plain[2, 1] = rand(K)
plain

[8477]
[1860]

In [4]:
key = zero_matrix(K, 2, 1)
key[1, 1] = rand(K)
key[2, 1] = rand(K)
key

[6908]
[9793]

In [5]:
nonce = rand(K)
nonce

4305

In [6]:
cipher = encrypt(plain, 
                 key, 
                 nonce, 
                 ciminion_2)

[7437]
[5786]

In [7]:
decrypt(cipher, 
        key, 
        nonce, 
        ciminion_2)

[8477]
[1860]

In [8]:
plain == decrypt(cipher, key, nonce, ciminion_2)

true

## Ciminion2 Polynomial model
Generate the Ciminion_2 polynomial model, and compute a DRL Gröbner basis.

In [9]:
polys = generate_Ciminion_2_polynomials(ciminion_2=ciminion_2,
                                        plain=plain,
                                        cipher=cipher);

sep = repeat("-", 70)
println(sep)

for i in 1:(rounds_C + rounds_E)
    for j in 1:3
        println(polys[3 * (i - 1) + j])
    end
    println(sep)
end

----------------------------------------------------------------------
10006*x_C_1__1 + 130*y_1 + y_2 + 6130
10006*x_C_2__1 + 1809*y_1 + 4368*y_2 + 1557
10006*x_C_3__1 + 131*y_1 + y_2 + 1602
----------------------------------------------------------------------
x_C_1__1*x_C_2__1 + x_C_3__1 + 10006*x_C_1__2 + y_1 + y_2 + 2318
8330*x_C_1__1*x_C_2__1 + x_C_1__1 + 8330*x_C_2__1 + 8330*x_C_3__1 + 10006*x_C_2__2 + 8330*y_1 + 8330*y_2 + 7078
x_C_1__1*x_C_2__1 + x_C_2__1 + x_C_3__1 + 10006*x_C_3__2 + y_1 + y_2 + 1151
----------------------------------------------------------------------
x_C_1__2*x_C_2__2 + x_C_3__2 + 10006*x_E_1__1 + y_1 + y_2 + 4937
5867*x_C_1__2*x_C_2__2 + x_C_1__2 + 5867*x_C_2__2 + 5867*x_C_3__2 + 10006*x_E_2__1 + 5867*y_1 + 5867*y_2 + 5005
x_C_1__2*x_C_2__2 + x_C_2__2 + x_C_3__2 + 10006*x_E_3__1 + y_1 + y_2 + 8506
----------------------------------------------------------------------
x_E_1__1*x_E_2__1 + x_E_3__1 + 10006*x_E_1__2 + y_1 + y_2 + 5255
5361*x_E_1__1*x_E_2__1 + 

Generate the efficient DRL term order which produces a quadratic DRL Gröbner basis.

In [10]:
P = parent(polys[1])
variables = gens(P)

variables_Q = String[]
for var in variables
    push!(variables_Q, string(var))
end

variables_Q = [
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 1:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 2] 
variables_Q[1:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E)];
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3]
]

Q, variables_Q = polynomial_ring(base_ring(P), 
                                 variables_Q, 
                                 internal_ordering=:degrevlex);

Generate a homomorphism between $P$ and $Q$.

In [11]:
variables_tmp = [
variables_Q[2 + 1:2 + 3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E)];
variables_Q[1:2];
variables_Q[3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3:3 * (ciminion_2.rounds_C - 1 + ciminion_2.rounds_E) + 3]
]

h = hom(P, Q, variables_tmp)

Ring homomorphism
  from multivariate polynomial ring in 18 variables over K
  to multivariate polynomial ring in 18 variables over K
defined by
  x_C_1__1 -> x_C_1__1
  x_C_2__1 -> x_C_2__1
  x_C_3__1 -> x_C_3__1
  x_C_1__2 -> x_C_1__2
  x_C_2__2 -> x_C_2__2
  x_C_3__2 -> x_C_3__2
  x_E_1__1 -> x_E_1__1
  x_E_2__1 -> x_E_2__1
  x_E_3__1 -> x_E_3__1
  x_E_1__2 -> x_E_1__2
  x_E_2__2 -> x_E_2__2
  x_E_3__2 -> x_E_3__2
  x_E_1__3 -> x_E_1__3
  x_E_2__3 -> x_E_2__3
  x_E_3__3 -> x_E_3__3
  y_1 -> y_1
  y_2 -> y_2
  x -> x

Now we compute the DRL Gröbner basis with respect to the efficient term order.

In [12]:
gb_Q = groebner_basis_f4(h(ideal(polys)), nr_thrds=16, info_level=2);


Legend for f4 information
--------------------------------------------------------
deg       current degree of pairs selected in this round
sel       number of pairs selected in this round
pairs     total number of pairs in pair list
mat       matrix dimensions (# rows x # columns)
density   density of the matrix
new data  # new elements for basis in this round
          # zero reductions during linear algebra
time(rd)  time of the current f4 round in seconds given
          for real and cpu time
--------------------------------------------------------

deg     sel   pairs        mat          density            new data         time(rd) in sec (real|cpu)
------------------------------------------------------------------------------------------------------
  1       2      12       3 x 6          66.67%        2 new       0 zero         0.02 | 0.19         
  2      11      11      19 x 25         24.63%       11 new       0 zero         0.04 | 0.46         
  2       5       5      36


--------------- INPUT DATA ---------------
#variables                      18
#equations                      18
#invalid equations               0
field characteristic         10007
homogeneous input?               0
signature-based computation      0
monomial order                 DRL
basis hash table resetting     OFF
linear algebra option            2
initial hash table size     131072 (2^17)
max pair selection             ALL
reduce gb                        1
#threads                        16
info level                       2
generate pbm files               0
------------------------------------------

---------------- TIMINGS ----------------
overall(elapsed)        0.14 sec
overall(cpu)            1.66 sec
select                  0.00 sec   0.0%
symbolic prep.          0.00 sec   0.0%
update                  0.00 sec   0.0%
convert                 0.00 sec   0.0%
linear algebra          0.00 sec   0.0%
reduce gb               0.00 sec   0.0%
--------------------------------

In [13]:
for poly in gb_Q
    println(poly)
    println(sep)
end

x_E_2__3 + 10006*x + 9203
----------------------------------------------------------------------
x_E_1__3 + 2989*x + 8637
----------------------------------------------------------------------
x_E_2__2 + 10006*x_E_3__3 + 7018*x + 4710
----------------------------------------------------------------------
x_E_1__2 + 4875*x_E_3__3 + 10006*x + 1009
----------------------------------------------------------------------
x_E_2__1 + 10006*x_E_3__2 + 5132*x_E_3__3 + x + 1846
----------------------------------------------------------------------
x_E_1__1 + 5361*x_E_3__2 + 10006*x_E_3__3 + 7018*x + 4796
----------------------------------------------------------------------
x_C_2__2 + 10006*x_E_3__1 + 4646*x_E_3__2 + x_E_3__3 + 2989*x + 8780
----------------------------------------------------------------------
x_C_1__2 + 5867*x_E_3__1 + 10006*x_E_3__2 + 5132*x_E_3__3 + x + 7058
----------------------------------------------------------------------
x_C_3__1 + 2362*x_C_3__2 + 8166*x_E_3__1 + 7645*